In [51]:
from datetime import datetime, timedelta
import requests
import logging
from pathlib import Path
import ast
import psycopg

try:
    import config as cfg
except ModuleNotFoundError:
    print("Создайте config.py с вашими данными! Как образец используйте config_example.py")

In [ ]:
class Attempt:
    def __init__(self, user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at):
        self.user_id = user_id
        self.is_correct = is_correct
        self.attempt_type = attempt_type
        self.created_at = created_at
        self.oauth_consumer_key = oauth_consumer_key
        self.lis_result_sourcedid = lis_result_sourcedid
        self.lis_outcome_service_url = lis_outcome_service_url

def clean_log_dir(log_dir, days = 3):
    cutoff_date = datetime.now() - timedelta(days=days)
    for log_file in log_dir.glob("*.log"):
        try:
            mtime = datetime.fromtimestamp(log_file.stat().st_mtime)
            if mtime < cutoff_date:
                log_file.unlink()
        except Exception as e:
            print(f"Не удалось удалить лог {log_file}: {e}")

def setup_logger():
    log_dir = Path("logs")

    clean_log_dir(log_dir)

    log_file = log_dir / f"{datetime.now().date() - timedelta(days = 1)}.log"

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )

def get_response():
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=1)

    api_url = cfg.api_url
    q_params = {
        'client': cfg.client,
        'client_key': cfg.client_key,
        'start': start_date,
        'end': end_date
    }
    logging.info("Начинаем скачивание данных с API")
    try:
        response = requests.get(api_url, params=q_params)
        response.raise_for_status() 
        logging.info("Данные успешно скачаны")
        return response.json()
    except requests.HTTPError as e:
        logging.error(f"Ошибка при обращении к API: {e.response.status_code}")
        return []
    except Exception as e:
        logging.error(f"Произошла непредвиденная ошибка: {e}")
        return []

def validate_attempt(attempt):
    required_keys = ['lti_user_id', 'is_correct', 'attempt_type', 'created_at']
    required_passback_keys = ['oauth_consumer_key', 'lis_result_sourcedid', 'lis_outcome_service_url']

    try:
        passback_params = ast.literal_eval(attempt.get('passback_params'))
    except (ValueError, SyntaxError, TypeError):
        return None
    
    if not all(key in attempt for key in required_keys):
        return None
    
    if not all(key in passback_params for key in required_passback_keys):
        return None
    
    if not isinstance(attempt.get('lti_user_id'), str):
        return None 
    
    attempt_type = attempt.get('attempt_type')
    is_correct = attempt.get('is_correct')

    if attempt_type == 'run':
        if is_correct is not None:
            return None
    elif attempt_type == 'submit':
        if is_correct != 1 and is_correct != 0:
            return None
        
    return passback_params
    
def create_attempts_objects(attempt, passback_params):
    user_id = attempt.get('lti_user_id')
    oauth_consumer_key = passback_params.get('oauth_consumer_key')
    lis_result_sourcedid = passback_params.get('lis_result_sourcedid')
    lis_outcome_service_url = passback_params.get('lis_outcome_service_url')
    is_correct = attempt.get('is_correct') if attempt.get('is_correct') is None else bool(attempt.get('is_correct'))
    attempt_type = attempt.get('attempt_type')
    created_at = attempt.get('created_at')
    attempt_o = Attempt(user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at)
    return attempt_o

def insert_data(conn, attempt_o):
    insert_query = """
    INSERT INTO user_attempts (
        user_id,
        oauth_consumer_key,
        lis_result_sourcedid,
        lis_outcome_service_url,
        is_correct,
        attempt_type,
        created_at
    )
    VALUES (
        %(user_id)s,
        %(oauth_consumer_key)s,
        %(lis_result_sourcedid)s,
        %(lis_outcome_service_url)s,
        %(is_correct)s,
        %(attempt_type)s,
        %(created_at)s
    )
    """

    with conn.cursor() as cur:
        cur.execute(insert_query, vars(attempt_o))
        
def main():
    setup_logger()
    attempts_data = get_response()
    valid_attempts = 0
    with psycopg.connect(**cfg.db_config) as conn:
        logging.info(f'Получено {len(attempts_data)} записей для обработки')
        logging.info('Начинаем проверку данных')
        for attempt in attempts_data:
            passback_params = validate_attempt(attempt)
            if not passback_params:
                user_id = attempt.get('lti_user_id', 'unknown')
                logging.warning(f'Пропущена попытка пользователя {user_id}: не прошла проверку')
                continue
            attempt_o = create_attempts_objects(attempt, passback_params)
            insert_data(conn, attempt_o)
            valid_attempts += 1 
        logging.info(f'В таблицу вставлено {valid_attempts} записей')
        logging.info(f'Не прошли проверку: {len(attempts_data) - valid_attempts} записей')

In [ ]:
main()